In [15]:
%matplotlib inline
import seaborn as sns
import numpy as np
import pandas as pd
import csv
import datetime
import matplotlib.pyplot as plt
from datapathfile import path_lookup_dir, datafile

In [16]:
def listcolumn(x):
    x = x.translate(None, '"')
    x = x.translate(None, ' ')
    x = x[1:-1].split(",")
    j = x[-1]
    return j

In [17]:
def TrueFalse(value):
    if value[0]=='F':
        value = False
    else:
        value = True
    return value

In [ ]:
reader = list(csv.reader(open(path_lookup_dir+datafile, 'rb'), delimiter='\t'))
colheaders = reader[0] 
df = pd.DataFrame(reader[1:])
dffilter = df.iloc[:, 0:20]
dffilter.columns = [colheaders]

In [ ]:
dffilter['installed'].unique()

In [ ]:
df.shape

In [ ]:
df = dffilter

In [ ]:
df['impression'] = df['impression'].apply(TrueFalse)

In [ ]:
df['installed'] = df['installed'].apply(TrueFalse)

In [ ]:
df['clicked'] = df['clicked'].apply(TrueFalse)

In [ ]:
df.ix[:, 'datetime'] = pd.to_datetime(df['datetime'], format="%Y-%m-%d %H:%M:%S")
df.ix[:, 'app_type'] = df['app_categories'].apply(listcolumn)


In [ ]:
df.shape

In [ ]:
drop_index=df[np.logical_and(
        np.logical_and(df['impression'], df['installed']),
        ~ df['clicked'])].index

In [ ]:
empty_index = np.where(df['app_type'] == '')
ul_index = np.where(df['app_type'] == 'ul')
df = df.drop(df.index[empty_index])
df = df.drop(df.index[ul_index])
df = df.drop(df.index[drop_index])

In [ ]:
df['weekday'] = df['datetime'].map(lambda x: x.isoweekday())
df['hours'] = df['datetime'].map(lambda x: x.hour)
df.ix[:, 'week'] = df['datetime'].map(lambda x:x.isocalendar()[1])

In [ ]:
impressiondf = df.groupby('city')['impression','clicked'].sum()
impression_index = impressiondf[impressiondf['impression'] > 5].index

In [ ]:
df=df[df.city.isin(impression_index)]

In [ ]:
filename='ATA_data.csv'

In [ ]:
df.to_csv(filename)